In [ ]:
# must run
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# checking the dir in our google drive
!ls /content/gdrive/MyDrive/Reporting_Data_Biases_In_Case_Reports/data

2010  2013	    new_2015.zip  new_2018.zip	new_2021.zip
2011  2022	    new_2016.zip  new_2019.zip	pmid_2010.list
2012  new_2014.zip  new_2017.zip  new_2020.zip	pmid_2011.list


In [ ]:
import os, re
import tarfile
from tqdm.auto import tqdm
import shutil

# shutil.copyfile(src, dst)

# 2nd option
# shutil.copy(src, dst) 

In [ ]:
def get_pdf_name(file_name):
  matches = re.match(r"^(PMC\d+)",file_name)
  if matches:
    return matches[0]

def get_pdf_name_from_url(url):
  matches = re.findall(r"[^\\/]+(?=)$",url)
  if matches:
    return matches[0]

url = 'ftp://ftp.ncbi.nlm.nih.gov\\pub/pmc/oa_pdf/41/99/main.PMC8207172.pdf'
print(os.path.basename(url))
print(get_pdf_name_from_url(url))

main.PMC8207172.pdf
main.PMC8207172.pdf


In [ ]:
!ls /content/gdrive/MyDrive/Reporting_Data_Biases_In_Case_Reports/data

2010  2013	    new_2015.zip  new_2018.zip	new_2021.zip
2011  2022	    new_2016.zip  new_2019.zip	pmid_2010.list
2012  new_2014.zip  new_2017.zip  new_2020.zip	pmid_2011.list


In [ ]:
# testing open tar file and extract pdf
root_dir = '/content/gdrive/MyDrive/Reporting_Data_Biases_In_Case_Reports/data'
# open file
year_s = '2011'
# one_file_name = '27876071.tar.gz'
new_dir = 'haotest/' + year_s +'/'
os.makedirs(new_dir, exist_ok=True)


one_dir = os.path.join(root_dir, year_s)

my_files = [afile for afile in os.listdir(one_dir) if afile.endswith('.tar.gz')]
my_pdf_files = [afile for afile in os.listdir(one_dir) if afile.endswith('.pdf')]
print(len(my_pdf_files))
print(len(my_files))
print(len(my_files)+ len(my_pdf_files))

676
996
1672


In [ ]:
no_pdf_zip_pmids=[]
yes_pdf_zip_pmids=[]
error_list_unzip = []

for one_file_name in tqdm(my_files):
    try:
    #     print(one_file_name)
        one_file_dir = os.path.join(one_dir, one_file_name)
    #     print(one_file_dir)

        pmid_name = one_file_name.replace('.tar.gz', '')
        tar = tarfile.open(one_file_dir)
        pdf_yes_flag = False
        for member in tar.getmembers():
          filename = member.name
          if filename.endswith('.pdf'):
    #         print('good match')
            pdf_yes_flag = True
            pmc_name = get_pdf_name(filename)
    #         print(pmc_name)
            tar.extract(member, os.path.join(new_dir, pmid_name +'.pdf'))
          elif filename.endswith('.nxml'):
    #         print('good match')
            pdf_yes_flag = True
            pmc_name = get_pdf_name(filename)
    #         print(pmc_name)
            tar.extract(member, os.path.join(new_dir, pmid_name +'.nxml'))
          # f=tar.extractfile(member)

        # extracting a specific file
        # tar.extract('*.pdf', '.')

        tar.close()
        if not pdf_yes_flag:
            print(f'no pdf found for {pmid_name}')
            no_pdf_zip_pmids.append(one_file_name)
        else:
            yes_pdf_zip_pmids.append(pmid_name)
    except Exception as e:
        print(f'{e} at: {pmid_name}')
        error_list_unzip.append(one_file_name)
        continue
        

print(len(yes_pdf_zip_pmids))
print(len(no_pdf_zip_pmids))
print(len(no_pdf_zip_pmids)+ len(yes_pdf_zip_pmids))
print(error_list_unzip)

  0%|          | 0/1006 [00:00<?, ?it/s]

1006
0
1006
[]


In [ ]:
error_list = []
for pdf_file in tqdm(my_pdf_files):
#     print(pdf_file)
    pdf_file_from_dir = os.path.join(one_dir, pdf_file)
    pdf_file_to_dir = os.path.join(new_dir, pdf_file)
#     print(pdf_file_from_dir)
#     print(pdf_file_to_dir)
    try:
#         shutil.copyfile(pdf_file_from_dir, pdf_file_to_dir) 
        shutil.copy2(pdf_file_from_dir, new_dir) 
#         os.rename(pdf_file_from_dir, pdf_file_to_dir)   # very fast
#         !mv $onefile_dir $moved_to_dir/  very slow
    except Exception as e:
        print(e)
        error_list.append(pdf_file)

print(error_list)

  0%|          | 0/521 [00:00<?, ?it/s]

[]


In [ ]:
!ls /content/gdrive/MyDrive/Reporting_Data_Biases_In_Case_Reports/data/

ls: cannot access '/content/gdrive/MyDrive/Reporting_Data_Biases_In_Case_Reports/data/haotest': No such file or directory


In [ ]:
year = 2010

one_dir = 'haotest/'+str(year)+'/'

moved_to_dir = 'haotest/new_'+ str(year)+'/'
os.makedirs(moved_to_dir, exist_ok=True)



my_files = [afile for afile in os.listdir(one_dir)]
print(len(my_files))


for one_file_dir in tqdm(my_files):
    target_file = get_pdf_name_from_url(one_file_dir)
    folder_dir = os.path.join(one_dir, one_file_dir)
#     print(folder_dir)
    for path, subdirs, files in os.walk(folder_dir):
        for name in files:
            file_url =os.path.join(path, name)
            print(f'moved from: {file_url}')
#             print(get_pdf_name_from_url(file_url))
            destination_file = os.path.join(moved_to_dir, target_file)
            print(f'moved to: {destination_file}')
            shutil.copy2(file_url, destination_file)

    

2520


  0%|          | 0/2520 [00:00<?, ?it/s]

moved from: haotest/2010/20191242.pdf/PMC5763681/TSWJ-2010-10-235760.pdf
moved to: haotest/new_2010/20191242.pdf
moved from: haotest/2010/21436949.nxml/PMC3063492/pamj-06-06.nxml
moved to: haotest/new_2010/21436949.nxml
moved from: haotest/2010/20360928.pdf/PMC2845778/cln_65p341.pdf
moved to: haotest/new_2010/20360928.pdf
moved from: haotest/2010/20952841.pdf/PMC2993987/IJO-58-530.pdf
moved to: haotest/new_2010/20952841.pdf
moved from: haotest/2010/20849656.nxml/PMC2949616/1471-2350-11-131.nxml
moved to: haotest/new_2010/20849656.nxml
moved from: haotest/2010/20103964.nxml/PMC2850188/ASM-30-81.nxml
moved to: haotest/new_2010/20103964.nxml
moved from: haotest/2010/21122256.pdf/PMC3294561/10-0678_finalL.pdf
moved to: haotest/new_2010/21122256.pdf
moved from: haotest/2010/20953549.pdf/PMC5763694/TSWJ-2010-10-527326.pdf
moved to: haotest/new_2010/20953549.pdf
moved from: haotest/2010/20689205.pdf/PMC2992925/IJO-58-437.pdf
moved to: haotest/new_2010/20689205.pdf
moved from: haotest/2010/205

In [ ]:
!ls haotest/2010/ | wc -l

2520


In [ ]:
!zip -q haotest/new_2010.zip haotest/new_2010/*

In [ ]:
!ls haotest/2014/ | wc -l
!ls haotest/2015/ | wc -l
!ls haotest/2016/ | wc -l
!ls haotest/2017/ | wc -l
!ls haotest/2018/ | wc -l
!ls haotest/2019/ | wc -l
!ls haotest/2020/ | wc -l
!ls haotest/2021/ | wc -l

5866
6611
7322
8570
10031
11720
18201
18499


In [ ]:
!ls haotest/

2014  2018  new_2014  simple_2014.zip  simple_2018.zip
2015  2019  new_2015  simple_2015.zip  simple_2019.zip
2016  2020  new_2016  simple_2016.zip  simple_2020.zip
2017  2021  new_2017  simple_2017.zip  simple_2021.zip


In [ ]:
!ls haotest/2014/ | wc -l
!ls haotest/2015/ | wc -l
!ls haotest/2016/ | wc -l
!ls haotest/2017/ | wc -l
!ls haotest/2018/ | wc -l
!ls haotest/2019/ | wc -l
!ls haotest/2020/ | wc -l
!ls haotest/2021/ | wc -l

5866
6611
7322
8570
10031
11720
18201
18499


In [ ]:
!ls haotest/new_2014/ | wc -l
!ls haotest/new_2015/ | wc -l
!ls haotest/new_2016/ | wc -l
!ls haotest/new_2017/ | wc -l
!ls haotest/new_2018/ | wc -l
!ls haotest/new_2019/ | wc -l
!ls haotest/new_2020/ | wc -l
!ls haotest/new_2021/ | wc -l

4864
5230
5561
5677
6349
8247
14915
15295


In [ ]:
!zip -q haotest/new_2014.zip haotest/new_2014/*
!zip -q haotest/new_2015.zip haotest/new_2015/*
!zip -q haotest/new_2016.zip haotest/new_2016/*
!zip -q haotest/new_2017.zip haotest/new_2017/*
!zip -q haotest/new_2018.zip haotest/new_2018/*
!zip -q haotest/new_2019.zip haotest/new_2019/*
!zip -q haotest/new_2020.zip haotest/new_2020/*
!zip -q haotest/new_2021.zip haotest/new_2021/*

In [ ]:
one_dir = 'haotest/2014/24673971.pdf'
print(get_pdf_name_from_url(one_dir))
for path, subdirs, files in os.walk(one_dir):
    for name in files:
        
        file_url =os.path.join(path, name)
        print(file_url)
        print(get_pdf_name_from_url(file_url))

24673971.pdf
haotest/2014/24673971.pdf/PMC4230632/1477-7819-12-68.pdf
1477-7819-12-68.pdf


In [ ]:
test_input_dir = 'haotest/2014/24673971.pdf/PMC4230632/'


pdf_file_from_dir = os.path.join(one_dir, pdf_file)
pdf_file_to_dir = os.path.join(new_dir, pdf_file)
#     print(pdf_file_from_dir)
#     print(pdf_file_to_dir)
try:
#         shutil.copyfile(pdf_file_from_dir, pdf_file_to_dir) 
    shutil.copy2(pdf_file_from_dir, new_dir) 
#         os.rename(pdf_file_from_dir, pdf_file_to_dir)   # very fast
#         !mv $onefile_dir $moved_to_dir/  very slow
except Exception as e:
    print(e)



In [ ]:
!ls 
# !unzip -q data_2016.zip
# !mv data/2016/ ./
# !rm -rf data

2018		pmid_2012.list	pmid_2019.list
2019		pmid_2013.list	pmid_2020.list
2020		pmid_2014.list	pmid_2021.list
2021		pmid_2015.list	pmid_2022.list
haotest		pmid_2016.list	process_unzip_case_reports_bridget.ipynb
pmid_2010.list	pmid_2017.list
pmid_2011.list	pmid_2018.list


In [ ]:
# !ls 2014 | wc -l
# !ls 2015 | wc -l

!ls 2017 | wc -l
!ls 2018 | wc -l
!ls 2019 | wc -l
!ls 2020 | wc -l
!ls 2021 | wc -l

5770
6895
7644
11441
10912


In [ ]:
# !rm -rf haotest/2021/*

In [ ]:
# !zip -q haotest/simple_2014.zip haotest/2014/*

In [ ]:
# !zip -q haotest/simple_2015.zip haotest/2015/*

In [ ]:
# !zip -q haotest/simple_2016.zip haotest/2016/*

In [ ]:
# !zip -q haotest/simple_2017.zip haotest/2017/*

In [ ]:
# !zip -q haotest/simple_2018.zip haotest/2018/*

In [ ]:
# !zip -q haotest/simple_2019.zip haotest/2019/*

In [ ]:
# !zip -q haotest/simple_2020.zip haotest/2020/*

In [ ]:
# !zip -q haotest/simple_2021.zip haotest/2021/*

In [ ]:
!ls 

2014	       data_2015.zip   pmid_2012.list  pmid_2017.list  pmid_2022.list
2015	       data_2016.zip   pmid_2013.list  pmid_2018.list  Untitled.ipynb
2016	       haotest	       pmid_2014.list  pmid_2019.list
2021	       pmid_2010.list  pmid_2015.list  pmid_2020.list
data_2014.zip  pmid_2011.list  pmid_2016.list  pmid_2021.list


In [ ]:
print(error_list)

[]


In [ ]:
!df -h

Filesystem                       Size  Used Avail Use% Mounted on
udev                              94G     0   94G   0% /dev
tmpfs                             19G  2.2M   19G   1% /run
/dev/mapper/aqua--vg-root        6.8T  6.2T  290G  96% /
tmpfs                             94G   20K   94G   1% /dev/shm
tmpfs                            5.0M  4.0K  5.0M   1% /run/lock
tmpfs                             94G     0   94G   0% /sys/fs/cgroup
/dev/sdd1                        6.9T  4.8T  1.8T  74% /projects
/dev/sdb1                        1.8T  211G  1.6T  12% /oldhome
/dev/sdc1                        1.8T  403G  1.4T  23% /public
tmpfs                             19G     0   19G   0% /run/user/1003
tmpfs                             19G     0   19G   0% /run/user/1001
tmpfs                             19G     0   19G   0% /run/user/1004
//elixr.dbmi.columbia.edu/Casey   15T   11T  3.9T  74% /projects/phi/ct2865/elixr/Casey
